In [ ]:
import numpy as np
import queue
from statistics import mean
import matplotlib.pyplot as plt
import math
import random

In [ ]:
arr_rate = 4
srv_rate = 4
duration = 3600
server_n = 4
job_size = 4
t = 0
jobs_iat = []
while t <= duration:
    iat = np.random.exponential(1/arr_rate)
    t += iat
    jobs_iat.append((iat,random.randint(1,4)))

i = 0
waiting_times = []
response_times = []
queue = []

servers = [] #[service_start,end]
for s in range(server_n):
    servers.append([0,0])

busy_times = []
for s in range(server_n):
    busy_times.append([])

jump_times = [[0,0]]
queue_size = [[0,0]]
t = 0

In [ ]:
def avl_server(t,servers):
    low = len(servers)
    i = 0
    for s in servers:
        if t > s[1]:
            if low == len(servers):
                low = i
            elif s[1] < servers[low][1]:
                low = i
        i += 1
    if low < len(servers):
        return low #return server with earliest idle time
    return len(servers)

def get_avl_servers(t,servers,n):
    empty = []
    i = 0
    for s in servers:
        if t >= s[1]: #get empty servers
            empty.append((i,s[1]))
        i += 1
    empty.sort(key = lambda x: x[1])
    return empty[:n] #return the [server index,free time]

def busy_server_n(t,servers):
    n = 0
    i = 0
    for s in servers:
        if t >= s[1] or t < s[0]: #get empty servers
            n += 1
            #print(i)
        i+=1
    return len(servers)-n #reverse




In [ ]:
for job in jobs_iat[:20]:
    t += jobs_iat[i][0]
    job_size = jobs_iat[i][1]
    st = np.random.exponential(1/srv_rate)
    #print(t)
    server_id = avl_server(t,servers) #get available server
    #there are available server, but there are jobs waiting in queue, and the avl servers is enough for next job
    if server_id != len(servers) and len(queue) > 0: 
        #fifoing the existing queue retroactively
        queue_id = 0
        next_job = queue[queue_id]
        avl_servers = len(servers)-busy_server_n(t,servers)
        while(server_id != len(servers) and len(queue) > 0 and queue_id < len(queue)):
            ready_servers = get_avl_servers(t,servers,pop_size)
            for s in ready_servers:
                curr_t = s[1]
                tmp_queue = []
                for q in queue:
                    if q[1] <= curr_t:
                        tmp_queue.append(q)
                
            if avl_servers < queue[queue_id][2]:
                queue_id += 1
                continue
            in_service = servers[server_id]
            popped = queue.pop(queue_id)
            pop_st = popped[0]
            start_queue = popped[1]
            pop_size = popped[2]
            ready_servers = get_avl_servers(t,servers,pop_size)
            start_service = ready_servers[-1][1]
            queue_size.append([start_service,len(queue)])
            waiting_times.append(start_service-start_queue)
            sids = ''
            #tmp_jump = []
            for s in ready_servers:
                server_id = s[0]
                sids = sids + str(server_id) + ','
                in_service = servers[server_id]
                if in_service[1] > 0:
                    print('Server '+str(server_id)+' is free at '+ str(round(in_service[1],3)))
                    print('Job (old) was served for '+str(round(in_service[1]-in_service[0],3)))
                    jump_times.append([in_service[1],busy_server_n(in_service[1],servers)])
                    #print(get_avl_servers(in_service[1],servers,pop_size))
                servers[server_id] = [start_service, start_service+pop_st]
                in_service = servers[server_id]
                busy_times[server_id].append(in_service)
            #tmp_jump.sort(key = lambda x: x[0]) 
            #jump_times += tmp_jump
            jump_times.append([in_service[0],busy_server_n(in_service[0],servers)])
            response_times.append(in_service[1]-start_queue)
            print('Server '+sids[:-1]+' is occupied at '+ str(round(in_service[0],3)))
            print('Job (new) was waiting in queue for '+str(round(in_service[0]-start_queue,3)))
            print('Queue size is now '+str(len(queue)))
            print('---------')
            server_id = avl_server(t,servers)
            avl_servers = len(servers)-busy_server_n(t,servers)
            queue_id = 0
        #jump_times.append([in_service[0],busy_server_n(in_service[0],servers)])
    server_id = avl_server(t,servers) #get available server
    avl_servers = len(servers)-busy_server_n(t,servers)
    if server_id != len(servers) and len(queue) == 0 and avl_servers >= job_size: #there are (still) available servers and queue is also empty
        last_jump = jump_times[-1][0]
        tmp_jump = []
        sn = 0
        for s in servers:
            if t >= s[1] and s[1] > 0 and s[1] > last_jump and s[1] != jump_times[-1][0]:
                tmp_jump.append([s[1],busy_server_n(s[1],servers)])
                print('Server '+str(sn)+' is free at '+ str(round(s[1],3)))
                print('Job was served for '+str(round(s[1]-s[0],3)))
                print('Queue size is now '+str(len(queue)))
                print('---------')
            sn += 1
        tmp_jump.sort(key = lambda x: x[0]) 
        jump_times += tmp_jump
        #get in service
        waiting_times.append(0)
        ready_servers = get_avl_servers(t,servers,job_size)
        start_service = ready_servers[-1][1]
        sids = ''
        for s in ready_servers:
            server_id = s[0]
            sids = sids + str(server_id) + ','
            in_service = servers[server_id]
            servers[server_id] = [t,t+st]
            in_service = servers[server_id]
            busy_times[server_id].append(in_service)
        jump_times.append([t,busy_server_n(t,servers)])
        response_times.append(st)
        print('Job of size '+str(job_size)+' arrived at: '+str(round(t,3)))
        print('Server '+sids[:-1]+' is occupied at '+ str(round(t,3)))
        print('Job did not wait in queue')
        print('---------')
    else: #server is busy, regardless of queue
        queue.append([st,t,job_size])
        queue_size.append([t,len(queue)])
        print('Job of size '+str(job_size)+' arrived at: '+str(round(t,3)))
        print('Job is put in queue')
        print('Queue size is now '+str(len(queue)))
        print('---------')
    i += 1
#queueing_times

In [ ]:
arr_rate = 4
srv_rate = 4
duration = 3600
server_n = 4
job_size = 4

t = 0
jobs_arr_t = []
while t <= duration:
    iat = np.random.exponential(1/arr_rate)
    t += iat
    jobs_arr_t.append((t,random.randint(1,4)))

jobs_arr_t

In [ ]:
waiting_times = []
response_times = []
queue = []

servers = [] #[service_start,end]
for s in range(server_n):
    servers.append([0,0])

busy_times = []
for s in range(server_n):
    busy_times.append([])

server_load = [[0,0]]
queue_size = [[0,0]]

job_id = 0
while job_id < len(jobs_arr_t) or len(queue) > 0:
    t = 999999999
    curr_job = ''
    if job_id < len(jobs_arr_t):
        curr_job = jobs_arr_t[job_id]
    for s in servers:
        if s[1] > 0 and s[1] < t and s[1] > server_load[-1][0]:
            t = s[1]
    if job_id < len(jobs_arr_t) and jobs_arr_t[job_id][0] < t:
        t = jobs_arr_t[job_id][0]
        queue.append(curr_job)
        job_id += 1
        print('Job arrive size '+str(curr_job[1]))
    print(t)
    
    queue_id = 0
    print(queue)
    while queue_id < len(queue):
        avl_servers_n = len(servers)-busy_server_n(t,servers)
        if avl_servers_n == 0:
            break
        job = queue[queue_id]
        if avl_servers_n >= job[1]:
            ready_servers = get_avl_servers(t,servers,job[1])
            st = np.random.exponential(1/srv_rate)
            for s in ready_servers:
                server_id = s[0]
                servers[server_id] = [t,t+st]
            waiting_times.append((t-job[0],job[1]))
            response_times.append((t+st-job[0],job[1]))
            queue.pop(queue_id)
            queue_id = 0
            print('Job start service size '+str(job[1]))
        else:
            queue_id += 1
    server_load.append((t,busy_server_n(t,servers)))
    queue_size.append((t,len(queue)))
    print('------------')

In [ ]:
server_load

In [ ]:
queue

In [ ]:
waiting_times

In [ ]:
response_times

In [ ]:
start_t = 0
end_t = 50

tmp_server = []
tmp_queue = []
init_sl = -1
for j in server_load:
    if j[0] < start_t:
        init_sl += 1
        continue
    if j[0] > end_t:
        break
    tmp_server.append(j)
if init_sl >= 0:
    tmp_server = [[start_t,server_load[init_sl][1]]] + tmp_server
tmp_server.append([end_t,tmp_server[-1][1]])

init_qs = -1
for q in queue_size:
    if q[0] < start_t:
        init_qs += 1
        continue
    if q[0] > end_t:
        break
    tmp_queue.append(q)
if init_qs >= 0:
    tmp_queue = [[start_t,queue_size[init_qs][1]]] + tmp_queue
tmp_queue.append([end_t,tmp_queue[-1][1]])

plt.figure(figsize=(17,5))
plt.step([q[0] for q in tmp_server], [q[1] for q in tmp_server], 'r', where='post', label = 'Server busy')
plt.step([q[0] for q in tmp_queue], [q[1] for q in tmp_queue], 'b', where='post', label = 'Queue size')
plt.legend()
plt.show()

In [ ]:
avg_wait = []
n = 1
tot = 0
for w in [wt[0] for wt in waiting_times]:
    tot += w
    avg_wait.append(tot/n)
    n += 1

avg_resp = []
n = 1
tot = 0
for w in [rt[0] for rt in response_times]:
    tot += w
    avg_resp.append(tot/n)
    n += 1

plt.figure(figsize=(15,10))
plt.plot([i for i in range(len(avg_wait))], avg_wait, label = "Avg Wait Times")
plt.plot([i for i in range(len(avg_resp))], avg_resp, label = "Avg Response Times")
plt.legend()
plt.show()

plt.bar

In [ ]:
boxed_wait = []
for i in range(server_n):
    job_size = i+1
    tmp_wait = []
    for wt in waiting_times:
        if wt[1] == job_size:
            tmp_wait.append(wt[0])
    boxed_wait.append(mean(tmp_wait))

boxed_resp = []
for i in range(server_n):
    job_size = i+1
    tmp_resp = []
    for rt in response_times:
        if rt[1] == job_size:
            tmp_resp.append(rt[0])
    boxed_resp.append(mean(tmp_resp))

width = 0.25
plt.bar([n+1 for n in range(server_n)], boxed_wait, label = 'Avg Waiting Times')
#plt.bar([n+1 for n in range(server_n)], boxed_resp, label = 'Avg Response Times')
plt.legend()
plt.xlabel('Job size')
plt.ylabel('time in seconds')
plt.show()

In [ ]:
print(mean([wt[0] for wt in waiting_times]))
print(mean([rt[0] for rt in response_times]))